In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import random

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading AI model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
import random

# Load model
print("Loading AI model...")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Simple example
text = "The moon was shining bright at night"
print(f"\nStarting text: '{text}'")
print("\nGenerating poem...\n")

base = text.split()
end = base[-1]
current_text = text + "\n"
lines = 4
length = len(base)
k = 5
greedy = False

for step in range(lines - 1):
    for i in range(length):
        # Encode current text
        input_ids = tokenizer.encode(current_text, return_tensors="pt")

        # Get predictions
        with torch.no_grad():
            outputs = model(input_ids)
            predictions = outputs.logits

        # Get the predictions for the NEXT token
        next_token_logits = predictions[0, -1, :]

        if i >= 0.75 * length:
            for w in ["and", "that", "which", "of", "to", "the", "a", "an", "in", "is", "has", "had", "have", "for",]:
                next_token_logits[tokenizer.encode(w)[0]] -= i / length * 2000


        next_token_probs = torch.softmax(next_token_logits, dim=0)

        # Get top k predictions
        top_probs, top_indices = torch.topk(next_token_probs, k)

        if(greedy):
          # Use the most likely word
          next_token_id = top_indices[0]
        else:
          #Random from top k
          next_token_id = top_indices[random.randint(0, k - 1)]

        next_word = tokenizer.decode([next_token_id])
        current_text += next_word
    current_text += "\n"

print(f"\nFinal generated text:\n'{current_text}'")

Loading AI model...


Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Starting text: 'The moon was shining bright at night'

Generating poem...


Final generated text:
'The moon was shining bright at night
It is said by its moonlight
It has no visible stars, so
The moon is a bright light.
'


In [ ]:
print(input_ids)

tensor([[  464,  8824,   373, 22751,  6016,   379,  1755,   198,  1026,   318,
           531,   416,   663,  8824,  2971,   198,  1026,   468,   645,  7424,
          5788,    11,   523,   198,   464,  8824,   318,   257,  6016,  1657]])
